In [1]:
from osgeo import gdal
from matplotlib import pyplot as plt
import numpy as np
import os
from src.data_utils import data_processing as dp
from src.data_utils.data_processing import make_model_dataset
from imblearn.ensemble import EasyEnsembleClassifier
import warnings
warnings.filterwarnings("ignore")
import xarray
import pandas as pd
from sklearn import preprocessing
from scipy import interpolate

In [2]:
f = xarray.open_dataset('sfcWind_day_inmcm4_rcp45_r1i1p1_20060101-20151231.nc')
df = f.to_dataframe()
df

time_bnds  lat_bnds  \
time                bnds lat    lon                                    
2006-01-01 12:00:00 0    -89.25 0.0    2006-01-01 00:00:00     -90.0   
                                2.0    2006-01-01 00:00:00     -90.0   
                                4.0    2006-01-01 00:00:00     -90.0   
                                6.0    2006-01-01 00:00:00     -90.0   
                                8.0    2006-01-01 00:00:00     -90.0   
...                                                    ...       ...   
2015-12-31 12:00:00 1     89.25 350.0  2016-01-01 00:00:00      90.0   
                                352.0  2016-01-01 00:00:00      90.0   
                                354.0  2016-01-01 00:00:00      90.0   
                                356.0  2016-01-01 00:00:00      90.0   
                                358.0  2016-01-01 00:00:00      90.0   

                                       lon_bnds  height  sfcWind  
time                bnds lat    lon                               
2006-01-01 12:00:00 0    -89.25 0.0        -1.0    10.0     0.62  
                                2.0         1.0    10.0     0.62  
                                4.0         3.0    10.0     0.62  
                                6.0         5.0    10.0     0.62  
                                8.0         7.0    10.0     0.62  
...                                         ...     ...      ...  
2015-12-31 12:00:00 1     89.25 350.0     351.0    10.0     1.14  
                                352.0     353.0    10.0     1.17  
                                354.0     355.0    10.0     1.19  
                                356.0     357.0    10.0     1.21  
                                358.0     359.0    10.0     1.23  

[157680000 rows x 5 columns]

In [3]:
a = df.loc[(df.lat_bnds >= 43.5 - 1.5) & (df.lat_bnds <= 47.0003 + 1.5)]
a = a.loc[(a.lon_bnds >= 36.5) & (a.lon_bnds <= 41.7502)]
a

time_bnds  lat_bnds  lon_bnds  \
time                bnds lat   lon                                             
2006-01-01 12:00:00 0    42.75 38.0  2006-01-01 00:00:00      42.0      37.0   
                               40.0  2006-01-01 00:00:00      42.0      39.0   
                               42.0  2006-01-01 00:00:00      42.0      41.0   
                         44.25 38.0  2006-01-01 00:00:00      43.5      37.0   
                               40.0  2006-01-01 00:00:00      43.5      39.0   
...                                                  ...       ...       ...   
2015-12-31 12:00:00 1    45.75 38.0  2016-01-01 00:00:00      46.5      39.0   
                               40.0  2016-01-01 00:00:00      46.5      41.0   
                         47.25 36.0  2016-01-01 00:00:00      48.0      37.0   
                               38.0  2016-01-01 00:00:00      48.0      39.0   
                               40.0  2016-01-01 00:00:00      48.0      41.0   

                                     height  sfcWind  
time                bnds lat   lon                    
2006-01-01 12:00:00 0    42.75 38.0    10.0    12.46  
                               40.0    10.0     8.05  
                               42.0    10.0     8.07  
                         44.25 38.0    10.0    13.65  
                               40.0    10.0     9.40  
...                                     ...      ...  
2015-12-31 12:00:00 1    45.75 38.0    10.0     5.12  
                               40.0    10.0     6.20  
                         47.25 36.0    10.0     5.20  
                               38.0    10.0     5.18  
                               40.0    10.0     5.69  

[109500 rows x 5 columns]

Reset Index

In [4]:
a = a.reset_index()
a.drop(columns=['time_bnds', 'lat_bnds', 'lon_bnds', 'height', 'bnds', 'time'], inplace=True)
a

,lat,lon,sfcWind
0,42.75,38.0,12.46
1,42.75,40.0,8.05
2,42.75,42.0,8.07
3,44.25,38.0,13.65
4,44.25,40.0,9.40
...,...,...,...
109495,45.75,38.0,5.12
109496,45.75,40.0,6.20
109497,47.25,36.0,5.20
109498,47.25,38.0,5.18


In [5]:
# Apply this trancform to the whole DataFrame a.
lat_idx = list(set(a.lat))
lon_idx = list(set(a.lon))
le_lat, le_lon = preprocessing.LabelEncoder(), preprocessing.LabelEncoder()
le_lat.fit(lat_idx)
le_lon.fit(lon_idx)

a.insert(loc=0, column='lat_idx', value=le_lat.transform(a.lat))
a.insert(loc=1, column='lon_idx', value=le_lon.transform(a.lon))
a.drop(columns=['lat', 'lon'], inplace=True)
a

,lat_idx,lon_idx,sfcWind
0,1,1,12.46
1,1,2,8.05
2,1,3,8.07
3,2,1,13.65
4,2,2,9.40
...,...,...,...
109495,3,1,5.12
109496,3,2,6.20
109497,4,0,5.20
109498,4,1,5.18


In [6]:
fin = []

for i in range(3650):
    aa = a[i*30:i*30+30]
    z = np.zeros((6, 4))
    for i, j, w in zip(aa['lat_idx'], aa['lon_idx'], aa['sfcWind']):
        z[i, j] = w
    fin.append(z[1:-1])

fin = np.stack(fin)

In [7]:
x = np.arange(36, 42.1, 2)       # longitude   25
y = np.arange(42.75, 47.26, 1.5) # latitude

xnew = np.arange(36, 42.1, 0.25)                         # 25
ynew = np.arange(42.75, 47.26, 0.25)#[::-1]              # 19

ss_x = np.where(((xnew >= 36.5) & (xnew <= 41.75)))[0]   # 22
ss_y = np.where(((ynew >= 43.5) & (ynew <= 47)))[0]      # 15

final = []
for z in fin:
    f = interpolate.interp2d(x, y, z)   # or (x, y, z)
    final.append(f(xnew, ynew)[np.ix_(ss_y, ss_x)])
final = np.stack(final)
final.shape

(3650, 15, 22)

In [8]:
new_final = []
for i in final:
    new_final.append(i[::-1])
final = np.stack(new_final)

In [ ]:
# Paste it in the beginning if you want to check if indices feb_28_in_2008_and_2012 are right
# time = pd.DataFrame(a.time.unique())      
# time.iloc[788]

In [10]:
feb_28_in_2008_and_2012 = [788, 2248]

feb_2012 = (final[2248] + final[2249]) / 2
feb_2012 = np.expand_dims(feb_2012, axis=0)
final = np.vstack((final[:2249], feb_2012, final[2249:]))

feb_2008 = (final[788] + final[789])/2
feb_2008 = np.expand_dims(feb_2008, axis=0)
final = np.vstack((final[:789], feb_2008, final[789:]))

Save the result

In [12]:
arr_reshaped = final.reshape(final.shape[0], -1)
  
# saving reshaped array to file.
np.savetxt("wind_in_box.txt", arr_reshaped)  

### Load data and trash

In [59]:
# retrieving data from file.
loaded_arr = np.loadtxt("wind_in_box.txt")
  
# This loadedArr is a 2D array, therefore, we need to convert it to the original
# array shape.reshaping to get original matrice with original shape.
lo = loaded_arr.reshape(
    3650, 15, 22) # (loaded_arr.shape[0], loaded_arr.shape[1] // final.shape[2], final.shape[2])

In [94]:
# pd.DataFrame(fin[0], index=[42.75, 44.25, 45.75, 47.25], columns=[36, 38, 40, 42])

,36,38,40,42
42.75,13.969999,12.46,8.05,8.07
44.25,13.540000,13.65,9.40,8.16
45.75,7.760000,6.84,6.66,5.63
47.25,2.300000,2.36,2.85,3.66


In [ ]:
# in_lat_y = [47.  , 46.75, 46.5 , 46.25, 46.  , 45.75, 45.5 , 45.25,  45.  , 44.75, 44.5 , 44.25, 44.  , 43.75, 43.5]
# pd.DataFrame(final[0], index=in_lat_y, columns=[36.5 , 36.75, 37.  , 37.25, 37.5 , 37.75, 38.  ,
#        38.25, 38.5 , 38.75, 39.  , 39.25, 39.5 , 39.75, 40.  , 40.25, 40.5 , 40.75, 41.  , 41.25, 41.5, 41.75])

x - 36.5 41.75    
y - 43.5 47       

y - широта Latitude    42.75

X - долгота Longitude  36

      42.75 44.25 45.75 47.25

36    13.9  13.5  7.7   
38    12.4  13.6  6.8   2.3
40    8.05  9.4   6.6   2.8
42          8.1   5.6   3.6
